<a href="https://colab.research.google.com/github/julwdo/thesis/blob/main/01_codes/01_gcolab/01_thesis_extract_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-17-jdk-headless -qq > /dev/null # OpenJDK 17
!wget --show-progress https://archive.apache.org/dist/spark/spark-3.5.6/spark-3.5.6-bin-hadoop3.tgz # Apache Spark 3.5.6 with Hadoop 3 support
!tar xf spark-3.5.6-bin-hadoop3.tgz
!pip install -q findspark
!pip install -q --upgrade pyspark==3.5.6

--2026-01-25 15:12:59--  https://archive.apache.org/dist/spark/spark-3.5.6/spark-3.5.6-bin-hadoop3.tgz
Resolving archive.apache.org (archive.apache.org)... 65.108.204.189, 2a01:4f9:1a:a084::2
Connecting to archive.apache.org (archive.apache.org)|65.108.204.189|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 400923510 (382M) [application/x-gzip]
Saving to: ‘spark-3.5.6-bin-hadoop3.tgz’

spark-3.5.6-bin-had 100%[===================>] 382.35M  14.1MB/s    in 26s     

2026-01-25 15:13:25 (14.7 MB/s) - ‘spark-3.5.6-bin-hadoop3.tgz’ saved [400923510/400923510]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.4/317.4 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 18.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dataproc-spark-connect 1.0.1 requ

In [2]:
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-17-openjdk-amd64'
os.environ['SPARK_HOME'] = '/content/spark-3.5.6-bin-hadoop3'

In [3]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = (
    SparkSession.builder
    .appName("BotDetection")
    .master("local[*]")
    .getOrCreate()
    )
spark

In [4]:
from google.colab import auth
import pyspark.sql.functions as F
from pyspark.sql.types import FloatType
from nltk.sentiment import SentimentIntensityAnalyzer
import math
from collections import Counter
from pyspark.sql import Window
from pyspark.sql.types import StringType, NumericType, BooleanType, ArrayType, FloatType

In [5]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [6]:
from google.colab import drive
drive.mount('/content/drive')

in_dir = "/content/drive/MyDrive/twibot-22/raw"

Mounted at /content/drive


In [7]:
users = spark.read.json(os.path.join(in_dir, "user.jsonl"))
#users.printSchema()

In [8]:
#users.show(5, truncate=False)

In [9]:
users_selected = users.select(
    F.col("id"),
    F.col("name"),
    F.col("username"),
    F.col("created_at"),
    F.col("description"),
    F.col("url"),
    F.col("entities.description.cashtags"),
    F.col("entities.description.hashtags"),
    F.col("entities.description.mentions"),
    F.col("entities.description.urls"),
    F.col("location"),
    F.col("pinned_tweet_id"),
    F.col("profile_image_url"),
    F.col("protected"),
    F.col("public_metrics.followers_count"),
    F.col("public_metrics.following_count"),
    F.col("public_metrics.listed_count"),
    F.col("public_metrics.tweet_count"),
    F.col("verified")
    )

In [10]:
#users_selected.printSchema()

In [11]:
#users_selected.show(5, truncate=False)

In [12]:
labels = spark.read.csv(os.path.join(in_dir, "label.csv"), header=True, inferSchema=True)

In [13]:
labels.show(5, truncate=False)

+--------------------+-----+
|id                  |label|
+--------------------+-----+
|u1217628182611927040|human|
|u2664730894         |human|
|u1266703520205549568|human|
|u1089159225148882949|human|
|u36741729           |bot  |
+--------------------+-----+
only showing top 5 rows



In [14]:
users_labeled = users_selected.join(labels, users_selected.id == labels.id, "left").drop(labels.id)

In [15]:
#users_labeled.show(5, truncate=False)

In [16]:
#print('Summary of missing values:')
#users_labeled.select([F.count(F.when(F.isnull(c), c)).alias(c) for c in users_labeled.columns]).show()

In [17]:
sia = SentimentIntensityAnalyzer()

def vader_sentiment(text):
  return sia.polarity_scores(text)["compound"]

vader_udf = F.udf(vader_sentiment, FloatType())

In [18]:
def shannon_entropy(string):
    if string.strip() == "":
        return 0.0
    counts = Counter(string)
    length = len(string)
    return -sum((count/length) * math.log2(count/length) for count in counts.values())

entropy_udf = F.udf(shannon_entropy, FloatType())

In [19]:
now = F.current_timestamp()

In [20]:
user_features_0 = users_labeled.select(
    F.col("id"),
    F.col("name"),
    F.col("username"),
    F.length(F.col("name")).alias("name_length"),
    F.length(F.col("username")).alias("username_length"),
    (F.length(F.col("username")) / F.greatest(F.length(F.col("name")), F.lit(1))).alias("username_name_length_ratio"),
    F.col("description"),
#    F.regexp_replace(F.regexp_replace(F.regexp_replace(F.col("description"), r"https?://t\.co/\S+", "<URL>"), r"(?<=^|\s)@\w+", "<USER>"), r"\b[\w\.-]+@[\w\.-]+\.\w+\b", "<EMAIL>").alias("description"),
    F.length(F.col("description")).alias("description_length"),
    F.when(F.col("name") == "", False).otherwise(True).alias("has_name"),
    F.when(F.col("username") == "", False).otherwise(True).alias("has_username"),
    F.when(F.col("description") == "", False).otherwise(True).alias("has_description"),
    F.when(F.col("url") == "", False).otherwise(True).alias("has_url"),
    F.when(F.col("location").isNull() | (F.col("location") == ""), False).otherwise(True).alias("has_location"),
    F.when(F.col("pinned_tweet_id").isNull(), False).otherwise(True).alias("has_pinned_tweet"),
    F.col("name").rlike("(?i)\\bbot\\b").alias("has_bot_word_in_name"),
    F.col("description").rlike("(?i)\\bbot\\b").alias("has_bot_word_in_description"),
    (F.length(F.regexp_replace(F.col("name"), "[^\\d]", "")) / F.greatest(F.length(F.col("name")), F.lit(1))).alias("ratio_digits_in_name"),
    (F.length(F.regexp_replace(F.col("username"), "[^\\d]", "")) / F.greatest(F.length(F.col("username")), F.lit(1))).alias("ratio_digits_in_username"),
    (F.length(F.regexp_replace(F.col("description"), "[^\\d]", "")) / F.greatest(F.length(F.col("description")), F.lit(1))).alias("ratio_digits_in_description"),
    (F.length(F.regexp_replace(F.col("name"), "[A-Za-z0-9 ]", "")) / F.greatest(F.length(F.col("name")), F.lit(1))).alias("ratio_special_chars_in_name"),
    (F.length(F.regexp_replace(F.col("username"), "[A-Za-z0-9 ]", "")) / F.greatest(F.length(F.col("username")), F.lit(1))).alias("ratio_special_chars_in_username"),
    (F.length(F.regexp_replace(F.col("description"), "[A-Za-z0-9 ]", "")) / F.greatest(F.length(F.col("description")), F.lit(1))).alias("ratio_special_chars_in_description"),
    (F.length(F.regexp_replace(F.col("name"), "[^A-Z]", "")) / F.greatest(F.length(F.regexp_replace(F.col("name"), "[^a-z]", "")), F.lit(1))).alias("name_upper_to_lower_ratio"),
    (F.length(F.regexp_replace(F.col("username"), "[^A-Z]", "")) / F.greatest(F.length(F.regexp_replace(F.col("username"), "[^a-z]", "")), F.lit(1))).alias("username_upper_to_lower_ratio"),
    entropy_udf(F.col("name")).alias("name_entropy"),
    entropy_udf(F.col("username")).alias("username_entropy"),
    (F.levenshtein(F.col("username"), F.col("name")) / F.greatest(F.length(F.col("username")), F.length(F.col("name")), F.lit(1))).alias("username_name_levenshtein"),
    vader_udf(F.col("description")).alias("description_sentiment"),
    F.when(F.col("cashtags").isNotNull(), F.size(F.col("cashtags"))).otherwise(F.lit(0)).alias("cashtag_in_description_count"),
    F.when(F.col("hashtags").isNotNull(), F.size(F.col("hashtags"))).otherwise(F.lit(0)).alias("hashtag_in_description_count"),
    F.when(F.col("mentions").isNotNull(), F.size(F.col("mentions"))).otherwise(F.lit(0)).alias("mention_in_description_count"),
    F.when(F.col("urls").isNotNull(), F.size(F.col("urls"))).otherwise(F.lit(0)).alias("url_in_description_count"),
    F.col("protected").alias("is_protected"),
    F.col("verified").alias("is_verified"),
    F.col("created_at"),
    (F.unix_timestamp(now) - F.unix_timestamp(F.to_timestamp("created_at"))).alias("account_age_seconds"),
    F.col("followers_count"),
    F.col("following_count"),
    F.col("listed_count"),
    F.col("tweet_count"),
    (F.col("followers_count") / F.greatest(F.col("following_count"), F.lit(1))).alias("followers_over_following"),
    (2 * F.col("followers_count") / F.greatest(F.col("following_count"), F.lit(1))).alias("double_followers_over_following"),
    (F.col("following_count") / F.greatest(F.col("followers_count"), F.lit(1))).alias("following_over_followers"),
    (F.col("following_count") / F.greatest(F.col("followers_count") ** 2, F.lit(1))).alias("following_over_followers_squared"),
    (F.col("following_count") / F.greatest(F.col("followers_count") + F.col("following_count"), F.lit(1))).alias("following_over_total_connections"),
    (F.col("listed_count") / F.greatest(F.col("followers_count"), F.lit(1))).alias("listed_over_followers"),
    (F.col("tweet_count") / F.greatest(F.col("followers_count"), F.lit(1))).alias("tweets_over_followers"),
    (F.col("listed_count") / F.greatest(F.col("tweet_count"), F.lit(1))).alias("listed_over_tweets"),
    (F.col("followers_count") / (F.unix_timestamp(now) - F.unix_timestamp(F.to_timestamp("created_at")))).alias("follower_rate"),
    (F.col("following_count") / (F.unix_timestamp(now) - F.unix_timestamp(F.to_timestamp("created_at")))).alias("following_rate"),
    (F.col("listed_count") / (F.unix_timestamp(now) - F.unix_timestamp(F.to_timestamp("created_at")))).alias("listed_rate"),
    (F.col("tweet_count") / (F.unix_timestamp(now) - F.unix_timestamp(F.to_timestamp("created_at")))).alias("tweet_rate"),
    F.col("label")
    )

In [21]:
tweets = spark.read.json(os.path.join(in_dir, "tweet_0.jsonl"))
#tweets.printSchema()

In [22]:
#tweets.show(5, truncate=False)

In [23]:
tweets_selected = tweets.select(
    F.col("id"),
    F.col("text"),
#    F.regexp_replace(F.regexp_replace(F.regexp_replace(F.col("text"), r"https?://t\.co/\S+", "<URL>"), r"(?<=^|\s)@\w+", "<USER>"), r"\b[\w\.-]+@[\w\.-]+\.\w+\b", "<EMAIL>").alias("text"),
    F.concat(F.lit("u"), F.col("author_id")).alias("author_id"),
    F.col("created_at"),
    F.col("in_reply_to_user_id"),
    F.when(F.col("in_reply_to_user_id").isNull(), False).otherwise(True).cast("int").alias("is_reply"),
    F.col("lang"),
    F.col("possibly_sensitive").cast("int").alias("is_sensitive"),
    F.col("public_metrics.like_count"),
    F.col("public_metrics.quote_count"),
    F.col("public_metrics.reply_count"),
    F.col("public_metrics.retweet_count")
    )

In [24]:
#tweets_selected.printSchema()

In [25]:
#tweets_selected.show(5, truncate=False)

In [26]:
#print('Summary of missing values:')
#tweets_selected.select([F.count(F.when(F.isnull(c), c)).alias(c) for c in tweets_selected.columns]).show()

In [27]:
# Step 1: count non-English tweets per user
non_en_count = tweets_selected.groupBy("author_id") \
    .agg(F.sum(F.when(F.col("lang") != "en", 1).otherwise(0)).alias("non_en_count"))

# Step 2: keep only users with zero non-English tweets
users_only_en = non_en_count.filter(F.col("non_en_count") == 0).select("author_id")

# Step 3: join back to filter tweets
tweets_only_en_users = tweets_selected.join(users_only_en, on="author_id", how="inner") \
    .drop("lang")

In [28]:
# Filter top 20 tweets per author
window = Window.partitionBy("author_id").orderBy(F.col("created_at").desc())
tweets_only_en_users = tweets_only_en_users.withColumn("rank", F.row_number().over(window))
tweets_filtered = tweets_only_en_users.filter(F.col("rank") <= 20).drop("rank")

In [29]:
user_features = (
    user_features_0
    .join(
        tweets_filtered.select("author_id").distinct(),
        user_features_0.id == tweets_filtered.author_id,
        how="left_semi"
        )
    )

In [30]:
tweet_features = (
    tweets_filtered
    .join(
        user_features.select("id", "label"),
        tweets_filtered.author_id == user_features.id,
        how="left"
    )
    .drop(user_features.id)
)

In [31]:
out_dir = "/content/drive/MyDrive/twibot-22/processed"

user_features.write.mode("overwrite").parquet(os.path.join(out_dir, "user_features_v1.parquet"))
#tweet_features.write.mode("overwrite").parquet(os.path.join(out_dir, "tweet_features_v1.parquet"))

In [33]:
edges = spark.read.csv(os.path.join(in_dir, "edge.csv"), header=True, inferSchema=True)

In [34]:
edges.printSchema()

root
 |-- source_id: string (nullable = true)
 |-- relation: string (nullable = true)
 |-- target_id: string (nullable = true)



In [35]:
users = user_features.select(F.col("id")).dropDuplicates()

In [36]:
edges_filtered = (
    edges
    .join(users.withColumnRenamed("id", "source_id"),
          on="source_id",
          how="inner")
    .join(users.withColumnRenamed("id", "target_id"),
          on="target_id",
          how="inner")
)

In [37]:
edges_filtered.write.mode("overwrite").parquet(os.path.join(out_dir, "edges_filtered.parquet"))

In [ ]:
user_data = (
    users_labeled
    .join(
        tweets_filtered.select("author_id").distinct(),
        users_labeled.id == tweets_filtered.author_id,
        how="left_semi"
        )
    )

user_data.write.mode("overwrite").parquet(os.path.join(out_dir, "user_data.parquet"))

In [ ]:
aggregated_tweet_features = tweet_features.groupBy("author_id").agg(
    F.collect_list("text").alias("top_tweets_text"),
    F.collect_list("created_at").alias("top_tweets_created_at"),
    F.collect_list("is_reply").alias("top_tweets_is_reply"),
    F.collect_list("is_sensitive").alias("top_tweets_is_sensitive"),
    F.collect_list("like_count").alias("top_tweets_like_count"),
    F.collect_list("quote_count").alias("top_tweets_quote_count"),
    F.collect_list("reply_count").alias("top_tweets_reply_count"),
    F.collect_list("retweet_count").alias("top_tweets_retweet_count")
    )

In [ ]:
enriched_user_features = user_features.join(
    aggregated_tweet_features,
    user_features.id == aggregated_tweet_features.author_id,
    how="left"
).drop("author_id").cache()

In [ ]:
enriched_user_features.write.mode("overwrite").parquet(os.path.join(out_dir, "enriched_user_features.parquet"))